In [1]:
import os
import gc
import psutil
from pathlib import Path
import pathlib

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup, DataCollatorWithPadding

from scipy.spatial.distance import cosine
from torch.utils.data import DataLoader, Dataset, SequentialSampler
from sklearn.neighbors import NearestNeighbors

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

In [2]:
class PATH:
    model = '/root/autodl-nas/model/sentence-transformers/all-MiniLM-L6-v2_new_r1.1'
    model_cpt = '/root/autodl-nas/model/f0r3/checkpoint-8708'
    # input_dir = '/root/autodl-nas/data/k12'
    input_dir = '/root/autodl-tmp/data/k12/cv_split_new/valid/fold_4'
    
    output_dir = '/root/autodl-tmp/data/k12/out'
    cv_dir = '/root/autodl-tmp/data/k12/cv_split_new/valid/fold_4'
    # pretrained_dir = '/root/autodl-nas/model/'
    content_dir = os.path.join(input_dir, 'content.csv')
    correlation_dir = os.path.join(input_dir, 'correlations.csv')
    submission_dir = os.path.join(input_dir, 'sample_submission.csv')
    topic_dir = os.path.join(input_dir, 'topics.csv')
    
    
class CFG:
    seed = 11
    fold = 1
    n_fold = 3
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'


## Lang

In [3]:
def get_level_features(df_topic, level_cols=['title']):
    df_hier = df_topic[list(set(level_cols + ['id', 'parent', 'level', 'has_content']))]
    highest_level = df_hier['level'].max()
    
    df_level = df_hier.query('level == 0').copy(deep=True)
    level_list = list()
    for col in level_cols:
        df_level[f'{col}_level'] = df_level[f'{col}'].apply(lambda x: [x])

    for i in tqdm(range(highest_level + 1)):
        level_list.append(df_level[df_level['has_content']])
        df_level_high = df_hier.query('level == @i+1')
        df_level = df_level_high.merge(df_level, left_on='parent', right_on='id', suffixes=['', '_parent'], how='inner')
        for col in level_cols:
            df_level[f'{col}_level'] = df_level[f'{col}_level'] + df_level[f'{col}'].apply(lambda x: [x])
        for col in df_level.columns:
            if col.endswith('_parent'):
                df_level.drop(columns=col, inplace=True)
    df = pd.concat(level_list).reset_index(drop=True)
    return df[list(set(['id'] + [f'{col}_level' for col in level_cols]))]

In [4]:
def get_topic_field(d):
    title = list(filter(lambda x: pd.notna(x), d['title_level']))
    title = ' of '.join(title[-1::-1])
    title = 'No information' if title=='' else title
    title = '[TITLE] ' + title + '. '
    description = d['description'] if pd.notna(d['description']) else 'No information'
    description = '[DESCRIPTION]' + description + '. '
    field = title + description
    return field

def get_content_field(d):
    title = d['title']
    title = 'No information' if pd.isna(title) else title
    title = '[TITLE] ' + title + '. '
    description = d['description'] if pd.notna(d['description']) else 'No information'
    description = '[DESCRIPTION]' + description + '. '
    kind = '[' + d['kind'] + '] '
    field = kind + title + description
    return field

In [5]:
def prepare_language_match(path, mode='train'):
    topic = pd.read_csv(path.topic_dir)[['id', 'language']]
    content = pd.read_csv(path.content_dir)[['id', 'language']]
    if mode == 'train':
        corr = pd.read_csv(path.correlation_dir)
    elif mode == 'valid':
        corr = pd.read_csv(path.submission_dir)
    
    topic = topic.merge(corr, left_on='id', right_on='topic_id', how='right')[['id', 'language']]
    match_dict = {}
    for language in topic['language'].unique():
        match_dict[language] = (topic.query('language==@language')[['id']], content.query('language==@language')[['id']])
    return match_dict

In [6]:
def prepare_match_features(topic, content, path):
    df_topic = pd.read_csv(path.topic_dir)
    df_content = pd.read_csv(path.content_dir)
    level = get_level_features(df_topic)
    df_topic = df_topic.merge(level, on='id', how='right')
    df_topic['field'] = df_topic.apply(lambda x: get_topic_field(x), axis=1)
    df_content['field'] = df_content.apply(lambda x: get_content_field(x), axis=1)
    topic = topic[['id']].merge(df_topic[['id', 'field']], on='id', how='left')
    content = content[['id']].merge(df_content[['id', 'field']], on='id', how='left')
    return topic, content

In [7]:
%%time
topic_content_match = prepare_language_match(PATH, mode='valid')

CPU times: user 9 s, sys: 944 ms, total: 9.95 s
Wall time: 9.95 s


In [8]:
# %%time
for lang in topic_content_match.keys():
    print(f'{lang}\t - topics: {len(topic_content_match[lang][0])}\t - contents: {len(topic_content_match[lang][1])}')
#     topic, content = topic_content_match[lang]
#     topic, content = prepare_match_features(topic, content, PATH)

en	 - topics: 3272	 - contents: 65939
es	 - topics: 1297	 - contents: 30844
gu	 - topics: 85	 - contents: 3677
pt	 - topics: 43	 - contents: 10435
hi	 - topics: 76	 - contents: 4042
fr	 - topics: 79	 - contents: 10682
bn	 - topics: 83	 - contents: 2513
fil	 - topics: 38	 - contents: 516
sw	 - topics: 17	 - contents: 1447
as	 - topics: 10	 - contents: 641


## Calc Embeddings

In [9]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

CPT_PATH = PATH.model_cpt
MODEL_PATH = PATH.model
OUTPUT_PATH = os.path.join(PATH.cv_dir)
N_NEIGHBOR = 100

In [10]:
class PlainDataset(Dataset):

    def __init__(self, df, tokenizer, label_name="") -> None:
        super().__init__()
        self.data = df[label_name].tolist()
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        text = self.data[index]
        inputs = self.tokenizer(
                text, 
                add_special_tokens = True,
                truncation='longest_first',
                max_length = 128,
                padding = 'max_length',
                return_attention_mask = True,
                return_tensors = 'pt',
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs

    def __len__(self):
        return len(self.data)

In [11]:
class Convert2Embed(object):

    def __init__(self) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        self.model = AutoModel.from_pretrained(CPT_PATH).cuda()

    def convert2embeddind(self, df, label_name=""):
        embed: list = []
        dataset = PlainDataset(df, tokenizer=self.tokenizer, label_name=label_name)
        dataloader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=32)
        for batch in dataloader:
            batch = {k: v.cuda() for k, v in batch.items()}
            with torch.no_grad():
                embeddings = self.model(**batch, output_hidden_states=True, return_dict=True).pooler_output
                embed.append(embeddings.cpu().clone().detach().numpy())
        embed = np.concatenate(embed, axis=0)
        return embed

    def get_embed(self):
        for lang in topic_content_match.keys():
            #
            # topic, content = topic_content_match[lang]
            # topic, content = prepare_match_features(topic, content, PATH)
            # topic_path = os.path.join(OUTPUT_PATH, "valid", f"topic_{lang}.pqt")
            # content_path = os.path.join(OUTPUT_PATH, "valid", f"content_{lang}.pqt")
            # topic.to_parquet(topic_path)
            # content.to_parquet(content_path)
            #
            
            for t in ["content", "topic"]:
                path = os.path.join(OUTPUT_PATH, "valid", f"{t}_{lang}.pqt")
                df = pd.read_parquet(path)
                embed = self.convert2embeddind(df, label_name=f"field")
                np.save(path.replace(".pqt", ".npy"), embed)


In [12]:
def valid():
#     with open(os.path.join(OUTPUT_PATH, "valid", "language.txt"), "r") as f:
#         valid_language = f.read().splitlines()
    recall_amount = 0
    recall_amount_total = 0
    recall_num = 0
    recall_total = {}
    f2_sum = 0
    df_list = []
    global df_pred_all
    for lang in topic_content_match.keys():
        ## debug
        # global df_pred, df_correlations
        global df_pred
        content_path = os.path.join(OUTPUT_PATH, "valid", f"content_{lang}.npy")
        topics_path = os.path.join(OUTPUT_PATH, "valid", f"topic_{lang}.npy")
        correlations_path = PATH.submission_dir
        content_array = np.load(content_path)
        topics_array = np.load(topics_path)
        model = NearestNeighbors(n_neighbors=N_NEIGHBOR, metric="cosine")
        model.fit(content_array)
        d, r = model.kneighbors(topics_array)
        df_content = pd.read_parquet(content_path.replace(".npy", ".pqt"))
        df_topics = pd.read_parquet(topics_path.replace(".npy", ".pqt"))
        df_correlations = pd.read_csv(correlations_path).astype({"topic_id": str})
        
        pred = {"topic_id": [], "content_ids": [], 'dists': []}
        for i in range(len(df_topics)):
            r_t = r[i]
            tmp = []
            for c in r_t:
                content_id = df_content.iloc[c]["id"]
                tmp.append(content_id)
            topics_id = df_topics.iloc[i]["id"]
            pred["topic_id"].append(topics_id)
            pred["content_ids"].append(tmp)
            pred['dists'].append(d[i])
        
        df_pred = pd.DataFrame(pred).astype({"topic_id": str})
        
        df_correlations['content_ids'] = df_correlations['content_ids'].apply(lambda x: list(x.split()))
        df_pred = df_pred.merge(df_correlations, on='topic_id', how='left', suffixes=['_pred', '_true'])
        df_pred['num'] = df_pred.apply(lambda x: len(x['content_ids_true']), axis=1)
        df_pred['hit'] = df_pred.apply(lambda x: len(set(x['content_ids_true']).intersection(x['content_ids_pred'])), axis=1)
        df_pred['recall'] = df_pred.apply(lambda x: x['hit'] / len(x['content_ids_true']), axis=1)
        df_pred['precision'] = df_pred.apply(lambda x: x['hit'] / len(x['content_ids_pred']), axis=1)
        df_pred['f2'] = 5*df_pred['precision']*df_pred['recall']/(4*df_pred['precision']+df_pred['recall'])
        recall = df_pred['recall'].mean()
        recall_total[lang] = recall
        recall_num += len(df_pred)
        recall_amount += df_pred['recall'].sum()
        recall_amount_total += df_pred['hit'].sum()/df_pred['num'].sum() * len(df_pred)
        f2_sum += df_pred['f2'].sum()
        df_list.append(df_pred)
        print(f"{lang}: {df_pred['hit'].sum()/df_pred['num'].sum()} - f2: {df_pred['f2'].sum()/len(df_pred)}")
        
    df_pred_all = pd.concat(df_list)
    print(f"Recall: total - {recall_amount_total/recall_num} line average - {recall_amount/recall_num} - f2 - {f2_sum/recall_num}")
    print(f"----------------Details----------------")
    for k, v in recall_total.items():
        print(f"Recall for language {k}: {v}")


In [13]:
%%time
P = Convert2Embed()
P.get_embed()

Some weights of the model checkpoint at /root/autodl-nas/model/f0r3/checkpoint-8708 were not used when initializing BertModel: ['lm_head.decoder.bias', 'lm_head.transform.LayerNorm.weight', 'lm_head.bias', 'lm_head.transform.dense.weight', 'lm_head.transform.dense.bias', 'lm_head.transform.LayerNorm.bias', 'lm_head.decoder.weight', 'mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/autodl-nas/model/f0r3/checkpoint-8708 and are newly initialized: ['bert.pooler.dense.weight',

CPU times: user 54.7 s, sys: 1min 10s, total: 2min 5s
Wall time: 2min 4s


In [14]:
def get_pred_by_thresh(df, thresh, n_default=1):
    pred_by_thresh = list()
    for i, d in enumerate(df['dists']):
        if d<=thresh:
            pred_by_thresh.append(df['content_ids_pred'][i])
    if not pred_by_thresh:
        pred_by_thresh = df['content_ids_pred'][:n_default]
    return pred_by_thresh

def calculate_f2(df_pred_all, true_label='content_ids_true', pred_label='pred_by_thresh') -> int:
    df_pred_all['hit'] = df_pred_all.apply(lambda x: len(set(x[pred_label]).intersection(x[true_label])), axis=1)
    df_pred_all['recall'] = df_pred_all.apply(lambda x: x['hit']/len(x[true_label]), axis=1)
    df_pred_all['precision'] = df_pred_all.apply(lambda x: x['hit']/len(x[pred_label]), axis=1)
    df_pred_all['f2'] = (5*df_pred_all['precision']*df_pred_all['recall']/(4*df_pred_all['precision']+df_pred_all['recall'])).fillna(0)
    return df_pred_all['f2'].mean()

def optimize_f2(df_pred_all, init_thresh=0.2, init_step=0.01) -> int:
    thresh = init_thresh
    df_pred_all['pred_by_thresh'] = df_pred_all.apply(lambda x: get_pred_by_thresh(x, 0.2), axis=1)
    f2 = calculate_f2(df_pred_all)

In [15]:
pd.read_csv(PATH.submission_dir).astype({"topic_id": str})

,topic_id,content_ids
0,t_0012a45fa09c,c_dde078b8ea7a
1,t_001bcbb22694,c_1d9dfc709413
2,t_00260f878951,c_86f126d7f1f8
3,t_003e944a4758,c_8f6966ad85f6
4,t_00535b89fd1d,c_189799d6b9a3 c_33e862f552f5 c_4f1702d3ffce c...
...,...,...
4995,t_ffba5459a977,c_04a421dba8aa c_787a7a2e7217 c_a46e0ec1377b c...
4996,t_ffc6ba0459d6,c_877d4e87d0e8
4997,t_ffc71a181765,c_d7be46af5e2b c_ea92fdd9899a
4998,t_ffdc013937fc,c_c27c5e711e25


In [16]:
# 275 383

In [17]:
# df_pred_all['pred_by_thresh'] = df_pred_all.apply(lambda x: get_pred_by_thresh(x, 0.4), axis=1)
# print(calculate_f2(df_pred_all))

# df_pred_all = df_pred_all[['topic_id', 'content_ids_pred', 'dists', 'content_ids_true']]
# # df_pred_all['labels'] = df_pred_all.apply(lambda x: label_pred(x), axis=1)
# for i in range(170, 450, 2):
#     thresh = i/1000
#     df_pred_all['pred_by_thresh'] = df_pred_all.apply(lambda x: get_pred_by_thresh(x, thresh, 3), axis=1)
#     f2 = calculate_f2(df_pred_all)
#     print(i, f2)

# df_pred_all.explode(['content_ids_pred', 'dists', 'labels'])

170 0.4024688386835773
172 0.4031549520762678
174 0.40409479083512645
176 0.40498780130113327
178 0.4056221467042033
180 0.40624726560527924
182 0.40721841030008704
184 0.409052690435736
186 0.41030785013923904
188 0.4105493930238
190 0.4121692194096475
192 0.4132842857948191
194 0.4144896801795744
196 0.41615635613829677
198 0.41766006453443766
200 0.41965067582784593
202 0.4202251120409235
204 0.42037800850695833
206 0.4214418561517112
208 0.4235262148368645
210 0.42532777889037154
212 0.42699839812736545
214 0.42848119762586157
216 0.43026228612064904
218 0.43200040637419174
220 0.4332824488732244
222 0.43483586366382354
224 0.4367978856794664
226 0.43781717992134717
228 0.4391767284118734
230 0.44009180614394006
232 0.4417736674332854
234 0.4430857341640051
236 0.44381462956587736
238 0.44525578177706604
240 0.4471523710828244
242 0.44870426476357084
244 0.4497284873353738
246 0.45163057757041214
248 0.4530941896947352
250 0.4548706386758581
252 0.45620181582970554
254 0.45695022797459456
256 0.4576392577793879
258 0.4577774966032547
260 0.45801281237339675
262 0.45868644081494886
264 0.458481439820201
266 0.45903038240513294
268 0.4595054445782289
270 0.4594602890972266
272 0.4594325829137616
274 0.45987696587445226
276 0.4602119501917246
278 0.46098732313376983
280 0.46121830781870066
282 0.46157880182145566
284 0.460916378887469
286 0.46063599518269166
288 0.46015462896328635
290 0.46026819515777456
292 0.46005084723586476
294 0.45913039790317867
296 0.45844901040602476
298 0.45750240076191034
300 0.45692206443176586
302 0.45608328843132245
304 0.4549603767536112
306 0.45329443276876796
308 0.4517214889907147
310 0.45083376456284796
312 0.44961959295741877
314 0.448034045429716
316 0.4461080456862716
318 0.44414396537646245
320 0.4425980612628054
322 0.44092277734608776
324 0.4389668410680323
326 0.4367121022990652
328 0.43420323722183946
330 0.4315968321759618
332 0.4294783204714232
334 0.4271767339470532
336 0.42463422374234044
338 0.42189842384377607
340 0.41922310125359186
342 0.4163151008820663
344 0.4132511312819868
346 0.4100375863146136
348 0.4068978474694664
350 0.4041449159496747
352 0.4010115542362085
354 0.3977875787529075
356 0.3948931385115175
358 0.3913653717224516
360 0.3877346850668936
362 0.38452566672457694
364 0.38094351006016375
366 0.37740241893768
368 0.3736814676972883
370 0.36987613243759815
372 0.36632612945386933
374 0.36227639122395994
376 0.35868013462650117
378 0.35507737565132824
380 0.3512169514101543
382 0.3472301898209479
384 0.343266006151315
386 0.3389849776348872
388 0.33505053612718283
390 0.33077591982865007
392 0.32659240791737504
394 0.3223306480778996
396 0.3180902050053218
398 0.3139647617144388
400 0.30977561550307586
402 0.3056369682307033
404 0.3015421054155209
406 0.29711072715398107
408 0.2929429504219797
410 0.2888552872196978
412 0.2845286862129868
414 0.28014803099027047
416 0.2759176750171542
418 0.2719160264420336
420 0.26765219196053575
422 0.2634692270432641
424 0.2592592306452557
426 0.25531781926012703
428 0.25119991693635824
430 0.24713812191438406
432 0.24296433979588497
434 0.23907443839564776
436 0.23522140710825673
438 0.2312847831007713
440 0.2273921890591881
442 0.22357551167977419
444 0.2195538619615006
​

In [18]:
%%time
N_NEIGHBOR = 6
valid()

en: 0.4571262662107233 - f2: 0.48277490540379264
es: 0.3604874919820398 - f2: 0.22549977668052346
gu: 0.2823920265780731 - f2: 0.2482648070883365
pt: 0.4533333333333333 - f2: 0.43505142642088673
hi: 0.3872832369942196 - f2: 0.3868624435269376
fr: 0.3446969696969697 - f2: 0.3901952908362572
bn: 0.34579439252336447 - f2: 0.299942092713177
fil: 0.5752212389380531 - f2: 0.5797437968322342
sw: 0.22321428571428573 - f2: 0.2570415571941125
as: 0.13043478260869565 - f2: 0.12142857142857144
Recall: total - 0.4238177989389195 line average - 0.5330710481221065 - f2 - 0.4049317689239942
----------------Details----------------
Recall for language en: 0.618578137888417
Recall for language es: 0.33973118674352293
Recall for language gu: 0.3055555555555555
Recall for language pt: 0.5859819121447029
Recall for language hi: 0.4614557226399332
Recall for language fr: 0.44811955852536633
Recall for language bn: 0.48572471705001824
Recall for language fil: 0.8159147869674186
Recall for language sw: 0.33969

In [19]:
%%time
N_NEIGHBOR = 5
valid()

en: 0.4182850506484289 - f2: 0.47988105834862155
es: 0.3296985246953175 - f2: 0.2241679934097922
gu: 0.2591362126245847 - f2: 0.2447586135900878
pt: 0.4 - f2: 0.4181193439716106
hi: 0.34104046242774566 - f2: 0.3633204565968043
fr: 0.32575757575757575 - f2: 0.392095481017385
bn: 0.3177570093457944 - f2: 0.3100972290527963
fil: 0.5575221238938053 - f2: 0.615464289692965
sw: 0.19642857142857142 - f2: 0.256348127076532
as: 0.13043478260869565 - f2: 0.13247863247863248
Recall: total - 0.3878662959955943 line average - 0.5019623385797884 - f2 - 0.40261932790433197
----------------Details----------------
Recall for language en: 0.5844308585350355
Recall for language es: 0.31403170311227363
Recall for language gu: 0.28679738562091506
Recall for language pt: 0.5428017718715392
Recall for language hi: 0.4105419799498746
Recall for language fr: 0.43217385534958136
Recall for language bn: 0.469569185834246
Recall for language fil: 0.8077694235588972
Recall for language sw: 0.32494972347913526
Reca

In [20]:
%%time
N_NEIGHBOR = 10
valid() # Recall: total - 0.5210813346754415 line average - 0.6065079786898661 - f2 - 0.3860233037535843

en: 0.5613265371959482 - f2: 0.4626540595943254
es: 0.4573444515715202 - f2: 0.21689126618047316
gu: 0.3488372093023256 - f2: 0.2480732095746388
pt: 0.5333333333333333 - f2: 0.4135193346693653
hi: 0.48265895953757226 - f2: 0.3921556352980062
fr: 0.42803030303030304 - f2: 0.39862724238605823
bn: 0.45794392523364486 - f2: 0.29770911556878954
fil: 0.5929203539823009 - f2: 0.45970822308609516
sw: 0.3125 - f2: 0.26885729420909854
as: 0.17391304347826086 - f2: 0.11399711399711399
Recall: total - 0.5241018208124375 line average - 0.6125549434677341 - f2 - 0.38863286081509746
----------------Details----------------
Recall for language en: 0.703240357124916
Recall for language es: 0.40644648656406557
Recall for language gu: 0.37333333333333335
Recall for language pt: 0.6464008859357696
Recall for language hi: 0.5566938178780285
Recall for language fr: 0.5269403695501983
Recall for language bn: 0.6112267250821468
Recall for language fil: 0.8240601503759398
Recall for language sw: 0.3980086906557

In [21]:
%%time
N_NEIGHBOR = 20
valid() # Recall: total - 0.6391622861430047 line average - 0.6950282194812909 - f2 - 0.32047399962118284


en: 0.6783018259242531 - f2: 0.3802976858182727
es: 0.5676715843489416 - f2: 0.18016762699994462
gu: 0.39867109634551495 - f2: 0.19693309141838555
pt: 0.6622222222222223 - f2: 0.3797461800625274
hi: 0.6040462427745664 - f2: 0.3415699152928255
fr: 0.5568181818181818 - f2: 0.3740049201591996
bn: 0.6261682242990654 - f2: 0.2562351679445053
fil: 0.672566371681416 - f2: 0.3261206994759627
sw: 0.5 - f2: 0.31540763158410223
as: 0.21739130434782608 - f2: 0.08467261904761905
Recall: total - 0.6392271533336713 line average - 0.6998912274884885 - f2 - 0.32129086048625716
----------------Details----------------
Recall for language en: 0.7880985677115436
Recall for language es: 0.4954159628600646
Recall for language gu: 0.41462184873949576
Recall for language pt: 0.7823827980804724
Recall for language hi: 0.6519789055973266
Recall for language fr: 0.6360284492265132
Recall for language bn: 0.7714044991153424
Recall for language fil: 0.8685463659147871
Recall for language sw: 0.5880288012640954
Reca

In [22]:
%%time
N_NEIGHBOR = 50 # 0.7559275033172171 line average - 0.7874367900873033 - f2 - 0.20525149868153938
valid()

en: 0.7901154913220207 - f2: 0.2397670645470287
es: 0.6779987171263631 - f2: 0.11664718417133597
gu: 0.5149501661129569 - f2: 0.13654847753215146
pt: 0.7511111111111111 - f2: 0.25180457217313895
hi: 0.7543352601156069 - f2: 0.22873269413427855
fr: 0.6893939393939394 - f2: 0.27686326409633744
bn: 0.8504672897196262 - f2: 0.16931051330554592
fil: 0.7876106194690266 - f2: 0.184673240480863
sw: 0.7321428571428571 - f2: 0.2695150716336357
as: 0.5652173913043478 - f2: 0.10808334951827722
Recall: total - 0.754219791533571 line average - 0.7894494128797634 - f2 - 0.20484645566703638
----------------Details----------------
Recall for language en: 0.8583047843474111
Recall for language es: 0.624432900261736
Recall for language gu: 0.5178057889822595
Recall for language pt: 0.8447120708748614
Recall for language hi: 0.7567721386800335
Recall for language fr: 0.7688679655470422
Recall for language bn: 0.8982278765411293
Recall for language fil: 0.9369806094182827
Recall for language sw: 0.84322344

In [23]:
%%time
N_NEIGHBOR = 100
valid() # 0.8259401867088655 line average - 0.8492008962245811 - f2 - 0.13077495168822426

en: 0.853022775662946 - f2: 0.1518439544717524
es: 0.758819756254009 - f2: 0.0752936705748993
gu: 0.584717607973422 - f2: 0.08836973858342756
pt: 0.8266666666666667 - f2: 0.1673585135697217
hi: 0.8179190751445087 - f2: 0.14647475112434108
fr: 0.7613636363636364 - f2: 0.18695092963253057
bn: 0.9252336448598131 - f2: 0.10273193879436954
fil: 0.8495575221238938 - f2: 0.10875709908763813
sw: 0.8839285714285714 - f2: 0.19345415133693683
as: 0.9130434782608695 - f2: 0.09409340659340659
Recall: total - 0.8232143570039386 line average - 0.8486067217656235 - f2 - 0.13039750669613046
----------------Details----------------
Recall for language en: 0.8997262417524368
Recall for language es: 0.7285239694668694
Recall for language gu: 0.5846311858076564
Recall for language pt: 0.8957456626061275
Recall for language hi: 0.8088659147869673
Recall for language fr: 0.8217167365603703
Recall for language bn: 0.9484118291347207
Recall for language fil: 0.9702216066481993
Recall for language sw: 0.91452991

In [24]:
%%time
N_NEIGHBOR = 200
valid()

en: 0.902703400219369 - f2: 0.08990645538675869
es: 0.832264271969211 - f2: 0.04504508261927148
gu: 0.6511627906976745 - f2: 0.053074321661303436
pt: 0.9066666666666666 - f2: 0.10304029970260217
hi: 0.8988439306358381 - f2: 0.08931900964664952
fr: 0.8352272727272727 - f2: 0.11696533454118821
bn: 0.9626168224299065 - f2: 0.057192037076095895
fil: 0.911504424778761 - f2: 0.0616730256893982
sw: 0.9642857142857143 - f2: 0.12155828943133964
as: 1.0 - f2: 0.05445500208263805
Recall: total - 0.8805300209915443 line average - 0.9002981366574633 - f2 - 0.07745390111353273
----------------Details----------------
Recall for language en: 0.934899798525425
Recall for language es: 0.8232946248700932
Recall for language gu: 0.656984126984127
Recall for language pt: 0.9448966408268735
Recall for language hi: 0.8683009607351714
Recall for language fr: 0.8718418718977171
Recall for language bn: 0.9656626506024097
Recall for language fil: 0.9824099722991689
Recall for language sw: 0.9331322272498743
Reca

shuffle?

groupby lang shuffle

In [25]:

!free -h

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
              total        used        free      shared  buff/cache   available
Mem:          755Gi        56Gi        34Gi       2.4Gi       664Gi       691Gi
Swap:         1.9Gi        37Mi       1.9Gi
